In [2]:
!pip install torch torchvision -q
!pip install opencv-python -q
!pip install -U ultralytics -q
!pip install scipy -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.9 MB/s eta 0:00:00


In [3]:
!pip install ultralytics kaggle --quiet

In [4]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ashikurrahmanbhuiyan","key":"846aeef3212d965aadd4597241c587bc"}'}

In [5]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d lylmsc/wider-face-for-yolo-training
!unzip wider-face-for-yolo-training.zip -d widerface_data

Streaming output truncated to the last 5000 lines.
  inflating: widerface_data/labels/wider_5499.txt  
  inflating: widerface_data/labels/wider_55.txt  
  inflating: widerface_data/labels/wider_550.txt  
  inflating: widerface_data/labels/wider_5500.txt  
  inflating: widerface_data/labels/wider_5501.txt  
  inflating: widerface_data/labels/wider_5502.txt  
  inflating: widerface_data/labels/wider_5503.txt  
  inflating: widerface_data/labels/wider_5504.txt  
  inflating: widerface_data/labels/wider_5505.txt  
  inflating: widerface_data/labels/wider_5506.txt  
  inflating: widerface_data/labels/wider_5507.txt  
  inflating: widerface_data/labels/wider_5508.txt  
  inflating: widerface_data/labels/wider_5509.txt  
  inflating: widerface_data/labels/wider_551.txt  
  inflating: widerface_data/labels/wider_5510.txt  
  inflating: widerface_data/labels/wider_5511.txt  
  inflating: widerface_data/labels/wider_5512.txt  
  inflating: widerface_data/labels/wider_5513.txt  
  inflating: wide

In [7]:
import os
import random
import shutil

src_images_dir = 'widerface_data/images'
src_labels_dir = 'widerface_data/labels'

dest_dir = 'widerface_subset'
dest_images_dir = os.path.join(dest_dir, 'images')
dest_labels_dir = os.path.join(dest_dir, 'labels')

os.makedirs(dest_images_dir, exist_ok=True)
os.makedirs(dest_labels_dir, exist_ok=True)

all_images = os.listdir(src_images_dir)
sample_size = int(0.05 * len(all_images))
sampled_images = random.sample(all_images, sample_size)

for img_name in sampled_images:
    label_name = img_name.replace('.jpg', '.txt')
    shutil.copy(os.path.join(src_images_dir, img_name), dest_images_dir)
    shutil.copy(os.path.join(src_labels_dir, label_name), dest_labels_dir)

print(f'Sampled {sample_size} images and labels copied.')


Sampled 644 images and labels copied.


In [8]:
import yaml
from sklearn.model_selection import train_test_split

all_images = os.listdir(dest_images_dir)
train_imgs, val_imgs = train_test_split(all_images, test_size=0.2, random_state=42)

def write_list_to_file(lst, filepath):
    with open(filepath, 'w') as f:
        for item in lst:
            f.write(f'{os.path.join(dest_images_dir, item)}\n')

write_list_to_file(train_imgs, 'train.txt')
write_list_to_file(val_imgs, 'val.txt')

data_yaml = {
    'train': 'train.txt',
    'val': 'val.txt',
    'nc': 1,
    'names': ['face']
}

with open('widerface_data.yaml', 'w') as f:
    yaml.dump(data_yaml, f)

print('Data yaml file created.')


Data yaml file created.


In [9]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')
model.train(data='widerface_data.yaml', epochs=10, imgsz=640, batch=16)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 98.9MB/s]


Ultralytics 8.3.162 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=widerface_data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, pr

100%|██████████| 755k/755k [00:00<00:00, 22.8MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

                   all        129       1914      0.649      0.379      0.411      0.189



10 epochs completed in 0.035 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.162 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:02<00:00,  1.86it/s]


                   all        129       1914      0.646      0.378      0.409      0.189
Speed: 0.3ms preprocess, 3.5ms inference, 0.0ms loss, 4.3ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e41905a3ed0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [11]:
from IPython.display import display, Image as IPyImage
from PIL import Image

sample_image_path = "/content/all.jpg"
results = model(sample_image_path)

for r in results:
    im_array = r.plot()
    im = Image.fromarray(im_array[..., ::-1])
    display(im)

Output hidden; open in https://colab.research.google.com to view.

In [19]:
model1 = YOLO('yolov8n_100e.pt')

test_image_path = '/content/all.jpg'
results = model1(test_image_path)

for r in results:
    im_array = r.plot()
    im = Image.fromarray(im_array[..., ::-1])
    display(im)